In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
# model_name = 'qanastek/51-languages-classifier'
model_name = 'huggingface/CodeBERTa-small-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at huggingface/CodeBERTa-small-v1 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# res = classifier("hello how are you ?")
# print(res)

In [3]:
# from transformers import AutoTokenizer

# # tokenizer = AutoTokenizer.from_pretrained("qanastek/51-languages-classifier")

# code_sample = "i am maninder')"

# tokens = tokenizer(code_sample, return_tensors="pt")
# print(tokens)



In [4]:
import datasets
data = datasets.load_from_disk('preprocessed_programming_language_data_count_great500')
data = data.shuffle(seed= 42)
# data = data.train_test_split(0.1)

In [5]:
data

Dataset({
    features: ['code', 'language_name'],
    num_rows: 66148
})

In [6]:
# data = datasets.load_dataset('pandas','preprocessed_programming_language_data.csv')
data = data.rename_column('code','text')
data = data.rename_column('language_name','label')

In [7]:
tokenized_datasets = data.map(lambda examples: tokenizer(examples['text'], truncation=True, padding=True), batched=True)
language_to_keep = ['COBOL','Go','C','C++','Java','Python','Perl','Kotlin','C#','Rust','Ruby',
'Fortran','JavaScript','Scala','ARM Assembly','Lua','Pascal','jq','AppleScript','Mathematica/Wolfram Language','Swift','Erlang','R','PowerShell',
         'Visual Basic .NET','PHP']

In [8]:
# tokenized_datasets['input_ids']
tokenized_datasets = tokenized_datasets.filter(lambda example: example['label'] in language_to_keep)
tokenized_datasets

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 26990
})

In [9]:
# labels = set(data["label"])
# # print(labels)
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label

In [10]:
# 'H' in labels

In [11]:
# set(data['language_name'])
data,tokenized_datasets

(Dataset({
     features: ['text', 'label'],
     num_rows: 66148
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 26990
 }))

In [12]:
# data[0]

In [13]:

label_mapping = {label: i for i, label in enumerate(set(tokenized_datasets['label']))}
# print("Label mapping:", label_mapping)

def map_labels(examples):
    if 'label' in examples:
        return {'label':label_mapping[examples['label']]}
    else:
        return {'label': []}  # or any other appropriate handling

tokenized_datasets = tokenized_datasets.map(map_labels)


Map:   0%|          | 0/26990 [00:00<?, ? examples/s]

In [14]:
label_mapping

{'Scala': 0,
 'JavaScript': 1,
 'COBOL': 2,
 'ARM Assembly': 3,
 'R': 4,
 'Lua': 5,
 'C++': 6,
 'Visual Basic .NET': 7,
 'Go': 8,
 'Erlang': 9,
 'C#': 10,
 'Rust': 11,
 'Ruby': 12,
 'Swift': 13,
 'Mathematica/Wolfram Language': 14,
 'PHP': 15,
 'Fortran': 16,
 'AppleScript': 17,
 'Pascal': 18,
 'Java': 19,
 'PowerShell': 20,
 'Python': 21,
 'C': 22,
 'Perl': 23,
 'Kotlin': 24,
 'jq': 25}

In [15]:
tokenized_datasets[0]['label'],label_mapping['AppleScript']

(14, 17)

In [16]:
id2label = {y: x for x, y in label_mapping.items()}

In [17]:
# pip install scikit-learn

In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate

accuracy = evaluate.load("accuracy")
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(set(tokenized_datasets['label'])), id2label=id2label, label2id=label_mapping,ignore_mismatched_sizes=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at huggingface/CodeBERTa-small-v1 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from sklearn.model_selection import train_test_split
tokenized_datasets_splited = tokenized_datasets.train_test_split(test_size=0.2, seed=42)

In [21]:
tokenized_datasets_splited

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 21592
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5398
    })
})

In [22]:
import accelerate
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=25,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_splited["train"],
    eval_dataset=tokenized_datasets_splited["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.831300,0.240185,0.944053
2,0.237200,0.211322,0.950722
3,0.129300,0.206144,0.953501
4,0.086000,0.214162,0.951834
5,0.074200,0.231554,0.953316
6,0.058100,0.241408,0.952946
7,0.046100,0.240300,0.952205
8,0.044100,0.247937,0.952390
9,0.038200,0.256519,0.953316
10,0.038600,0.254281,0.950167


Attempted to log scalar metric loss:
0.8313
Attempted to log scalar metric learning_rate:
1.9407407407407407e-05
Attempted to log scalar metric epoch:
0.74
Attempted to log scalar metric eval_loss:
0.24018527567386627
Attempted to log scalar metric eval_accuracy:
0.9440533530937384
Attempted to log scalar metric eval_runtime:
81.2931
Attempted to log scalar metric eval_samples_per_second:
66.402
Attempted to log scalar metric eval_steps_per_second:
2.079
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric loss:
0.2372
Attempted to log scalar metric learning_rate:
1.8814814814814816e-05
Attempted to log scalar metric epoch:
1.48
Attempted to log scalar metric eval_loss:
0.21132202446460724
Attempted to log scalar metric eval_accuracy:
0.9507224898110411
Attempted to log scalar metric eval_runtime:
81.2253
Attempted to log scalar metric eval_samples_per_second:
66.457
Attempted to log scalar metric eval_steps_per_second:
2.081
Attempted to log scalar metric epoch:
2.

Checkpoint destination directory my_awesome_model/checkpoint-675 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_model/checkpoint-1350 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=16875, training_loss=0.0703159267284252, metrics={'train_runtime': 25216.2087, 'train_samples_per_second': 21.407, 'train_steps_per_second': 0.669, 'total_flos': 7.15365067444224e+16, 'train_loss': 0.0703159267284252, 'epoch': 25.0})

In [23]:
trainer.save_model("./trained_model/")

In [24]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


## Inference Code for the script

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
model_name = './trained_model/'
loaded_tokenizer = AutoTokenizer.from_pretrained(model_name)
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [27]:
loaded_model.config


RobertaConfig {
  "_name_or_path": "./trained_model/",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Scala",
    "1": "JavaScript",
    "2": "COBOL",
    "3": "ARM Assembly",
    "4": "R",
    "5": "Lua",
    "6": "C++",
    "7": "Visual Basic .NET",
    "8": "Go",
    "9": "Erlang",
    "10": "C#",
    "11": "Rust",
    "12": "Ruby",
    "13": "Swift",
    "14": "Mathematica/Wolfram Language",
    "15": "PHP",
    "16": "Fortran",
    "17": "AppleScript",
    "18": "Pascal",
    "19": "Java",
    "20": "PowerShell",
    "21": "Python",
    "22": "C",
    "23": "Perl",
    "24": "Kotlin",
    "25": "jq"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "ARM Assembly": 3,
    "AppleScript": 17,
    "C": 22,
    "C#": 10,
    "

In [28]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
text = """
tell application "Slack" to activate
tell application "Slack" to quit

"""
inputs = loaded_tokenizer(text, return_tensors="pt",truncation=True)
# inputs.to(device)
with torch.no_grad():
    logits = loaded_model(**inputs).logits
predicted_class_id = logits.argmax().item()
loaded_model.config.id2label[predicted_class_id]


'AppleScript'